In [3]:
import requests, re
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [4]:
headers = {'Accept-Language': 'en-US,en;q=0.8'}
result = requests.get('https://dubawa.org/false-no-evidence-to-support-ritualistic-claim-against-cdc-chairman-in-charloes-murder/', headers=headers)

src = result.content     
soup = BeautifulSoup(src, 'html.parser')  

#title = soup.find('h1', class_ = "hero--page-title page-title").text
#print(title)

In [55]:
claim =  re.findall(r'Claim:\s(.+)', str((soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
verdict = re.findall(r'Verdict:\s(.+)', str((soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background').text).strip()))[0]
label = re.findall(r'(\w+)\.', str(verdict))[0]

claim, verdict, label


('The family of Justice Gloria Musu-Scott, in a recent statement, quotes the ruling party Chairman Mulbah Morlu as saying, “their daughter Charloe Musu was killed for ritualistic purposes.”',
 'False. Online research on the particular Press Conference revealed that Morlu did not mention such a statement. The ruling party chairman also openly clarified the allegation to the media.',
 'False')

In [63]:
text = soup.find('div', class_ = 'entry-content entry clearfix')
full_text_heading = text.find('h2', text='Full Text') #find the Full Text heading
full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') # extract all the paragraphs after the Full Text heading

full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])

full_text

'The murder and armed robbery attacks on the home of Liberia’s former Chief Justice, Gloria Musu-Scott, have been greeted with several accusations, one of which is coming from the family of Justice Scott.\nCharloe Musu, daughter of Justice Scott, was murdered on the morning of Thursday,\xa0 February 23, 2023, at Justice Scott’s residence in Brewville on the outskirt of the city.\nOther members of the household suffered injuries in the attack. It is not yet clear the motive for the murder and the attack but the news of the murder has since gone viral.\xa0\xa0\nThe police have since begun an investigation into the matter.\nWhile citizens look forward to a speedy investigation and the arrest of the culprits, there have been some allegations and counter-allegations. One such is from the family of the former Chief Justice.\xa0\nOn March 6, 2023, the family issued a statement accusing the ruling party chairman, Mulbah Morlu, of making statements against the family.\nThe statement partly read

In [106]:
full_text[0]

'T'

In [64]:
author = soup.find('a', class_ = 'author-name tie-icon').text
date = soup.find('span', class_ = 'date meta-item tie-icon').text
#title = soup.find('h1', class_ = 'post-title entry-title').text
#author, date

('Varney Dukuly', 'March 22, 2023')

In [71]:
date = soup.find('span', class_ = 'date meta-item tie-icon').text
date_obj = datetime.strptime(date, '%B %d, %Y')
formatted_date = date_obj.strftime('%d/%m/%Y')
formatted_date

'22/03/2023'

## Link extraction 

In [2]:
import requests
from bs4 import BeautifulSoup
import time
from langdetect import detect

def is_english(text):
    try:
        lang = detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

start_time = time.time()

fc_links = []
pages_NG = []
for i in range (1, 78):
    page = f"https://dubawa.org/category/fact-check/page/{i}/"
    pages_NG.append(page)
    
for page in pages_NG:
    url = page
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    aux = [] # lista auxiliar para reducir el coste temporal de buscar en la lista a continuación
    links = soup.find_all("h2", {"class": "post-title"})
    for link in links:
        link = str(link).split('href=')[1][1:].split('">')[0]
        if is_english(link.split("/")[:-1][-1]) and not link in aux:
            aux.append(link)
    fc_links = fc_links + aux
    
print(len(fc_links))
print('\n')
for i in fc_links:
    print(i)

print('\n')
print("--- %s seconds ---" % (time.time() - start_time))


714


https://dubawa.org/false-peter-obi-did-not-travel-to-london-over-health-issues/
https://dubawa.org/african-fact-checking-awards-opens-for-entry/
https://dubawa.org/news-headlines-of-world-leaders-condemning-nigerian-polls-fake/
https://dubawa.org/fact-checking-apc-supporters-claim-on-nigerias-debt-servicing-in-2022/
https://dubawa.org/inquirer-newspaper-managing-editors-statement-that-radio-invention-precedes-newspaper-is-untrue/
https://dubawa.org/false-uk-government-did-not-issue-apology-to-peter-obi-for-detaining-him/
https://dubawa.org/hoax-ecobank-is-not-running-any-subsidy-reward-scheme/
https://dubawa.org/is-brahmi-medication-permanent-cure-for-hypertension/
https://dubawa.org/hoax-bola-tinubu-not-giving-n25000-presidential-election-funds/
https://dubawa.org/fact-checking-claims-by-peter-obi-in-post-election-interview/
https://dubawa.org/old-video-from-nigeria-used-to-make-homosexuality-claim-in-uganda/
https://dubawa.org/has-inec-uploaded-results-of-national-assembly-elec

In [390]:
fallos = []
def get_data_from_url(url):
    result = requests.get(f'{url}')
    src = result.content     
    soup = BeautifulSoup(src, 'html.parser')
    try:
        try:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
        except IndexError:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]
        except:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p').text.strip()))[0]

        verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')

        if verdict == [] or verdict is None or verdict.text[:7] == 'FINDING':
            try:
                label = re.findall(r'[VerdictVERDICT]?:\s(\w+)', (soup.find('p', class_ = 'has-vivid-red-background-color has-background').text))
            except:
                label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', label_raw[0].text, flags=re.IGNORECASE)[0]


            if label is None or label == []:
                obj = soup.find('figure', class_ = 'alignright size-full')
                if obj is None:
                    obj = soup.find('figure', class_ = 'alignright size-large')
                try:
                    label = re.findall(r'\w+\-?\w+', str(obj.img['data-image-title']))[0]
                    if (label[-1:]).isdigit():
                        label = label[:-2]
                except:
                    label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                    label = re.findall(r'[VERDICT]?:\s|\b(\w+)', (label_raw[0]).text)

        else: 
            try:
                label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
            except (AttributeError or IndexError): 
                verdict = soup.find('p', class_='has-white-background-color has-background')
                label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
                
        label = label.capitalize()
        if '-' in label:
            label.replace('-', ' ')
        
        text = soup.find('div', class_ = 'entry-content entry clearfix')
        try:
            full_text_heading = soup.find('h2', text=['Full Text', 'FULL TEXT', 'Full text'])
            full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 

            if (full_text_paragraphs == '') or (full_text_paragraphs == []):
                full_text_paragraphs = full_text_heading.find_all_next('p')
            full_text = ''
            for p in full_text_paragraphs:
                if 'The researcher' in p.text:
                    full_text += p.text[:(p.text).index('The researcher')]
                    break
                elif 'Is there a claim' in p.text:
                    full_text += p.text[:(p.text).index('Is there a claim')]
                    break 
                else:
                    full_text += p.text + '\n'
        except:
            full_text = ''
            finding_text = None
            for p in soup.find_all('p'):

                if "FINDING:" in p.text:
                    finding_text = p
                    full_text_paragraphs = finding_text.find_all_next('p')

                    for p in full_text_paragraphs:
                        if 'VERDICT' in p.text:
                            full_text += p.text[:(p.text).index('VERDICT')]
                            break

                        else:
                            full_text += p.text + '\n'

                    if full_text == '':
                        full_text = finding_text.text

                elif 'Is there a claim' in p.text:
                    full_text += p.text[:(p.text).index('Is there a claim')]
                    break 

                elif finding_text is  None:
                    full_text += p.text + '\n'
                    
        clean_text = full_text.replace('\n', ' ').replace('\xa0', ' ')
                    
        date = soup.find('span', class_ = 'date meta-item tie-icon').text
        date_obj = datetime.strptime(date, '%B %d, %Y')
        formatted_date = date_obj.strftime('%d/%m/%Y')
        
        yield {"Claim": claim, "Evidences": clean_text, "Label": label, "Date": formatted_date, "Url": url}        
        
        
    except:
        fallos.append(url)

# We use a generator to provide the results calling the function created 
results = list((get_data_from_url(url) for url in fc_links))
res_list = []

for result in results:
    try:
        res_list.append(next(result))
    except (StopIteration or AttributeError):
        pass

In [391]:
len((res_list))

435

In [392]:
len((fallos))

965

In [305]:
fallos

['https://dubawa.org/enugus-gubernatorial-election-suspended-no-winner-announced/',
 'https://dubawa.org/true-inec-collation-officer-in-abia-state-brutalised-by-thugs/',
 'https://dubawa.org/doctored-statement-of-inecs-presiding-officer-for-abia-state-circulates-whatsapp/',
 'https://dubawa.org/no-evidence-police-colluded-with-thugs-inec-to-disrupt-rivers-state-election/',
 'https://dubawa.org/false-no-evidence-to-support-ritualistic-claim-against-cdc-chairman-in-charloes-murder/',
 'https://dubawa.org/president-bio-commissioned-a-new-terminal-not-a-new-airport/',
 'https://dubawa.org/false-ngige-did-not-warn-apc-against-celebrating-the-presidential-election-victory/',
 'https://dubawa.org/nigerias-finance-minister-lied-in-claiming-countrys-vat-rate-lowest-in-the-world/',
 'https://dubawa.org/us-envoys-comment-about-disinformation-from-some-liberians-in-us-not-fake/',
 'https://dubawa.org/did-adeboye-lead-protest-against-jonathan-in-2014-as-twitter-user-claims/',
 'https://dubawa.org/d

In [102]:
len(res_list), len(fc_links)

(6, 710)

In [112]:
url

'https://dubawa.org/category/fact-check/page/77/'

In [241]:
('FALSE-5'[:-2])

'FALSE'

In [ ]:
import re
labels, claims, textos = [], [], []
textos_fallo = 0
for i in fc_links:
    result = requests.get(f'{i}')
    src = result.content     
    soup = BeautifulSoup(src, 'html.parser')
    try:
        try:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
        except IndexError:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]
        except:
            claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p').text.strip()))[0]

        verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')

        if verdict == [] or verdict is None or verdict.text[:7] == 'FINDING':

            try:
                label = re.findall(r'[VerdictVERDICT]?:\s(\w+)', (soup.find('p', class_ = 'has-vivid-red-background-color has-background').text))
            except:
                label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', label_raw[0].text, flags=re.IGNORECASE)[0]

            if label is None or label == []:
                obj = soup.find('figure', class_ = 'alignright size-full')
                if obj is None:
                    obj = soup.find('figure', class_ = 'alignright size-large')
                try:
                    label = re.findall(r'\w+\-?\w+', str(obj.img['data-image-title']))[0]
                    if (label[-1:]).isdigit():
                        label = label[:-2]

                except:
                    label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                    label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', label_raw[0].text, flags=re.IGNORECASE)[0]

        else: 
            try:
                label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', str((verdict.text).strip()), flags=re.IGNORECASE)[0]
                #label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
            except (AttributeError or IndexError): 
                verdict = soup.find('p', class_='has-white-background-color has-background')
                #label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
                label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', str((verdict.text).strip()), flags=re.IGNORECASE)[0]


        label = label.capitalize()
        if '-' in label:
            label.replace('-', ' ')

        text = soup.find('div', class_ = 'entry-content entry clearfix')

        try:
            full_text_heading = soup.find('h2', text=['Full Text', 'FULL TEXT', 'Full text'])

            full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 

            if (full_text_paragraphs == '') or (full_text_paragraphs == []):
                full_text_paragraphs = full_text_heading.find_all_next('p')


            full_text = ''
            for p in full_text_paragraphs:
                if 'The researcher' in p.text:
                    full_text += p.text[:(p.text).index('The researcher')]
                    break
                elif 'Is there a claim' in p.text:
                    full_text += p.text[:(p.text).index('Is there a claim')]
                    break 
                else:
                    full_text += p.text + '\n'
                clean_text = full_text.replace('\n', ' ').replace('\xa0', ' ')
        except:
                full_text = ''
                finding_text = None
                for p in soup.find_all('p'):

                    if "FINDING:" in p.text:
                        finding_text = p
                        full_text_paragraphs = finding_text.find_all_next('p')

                        for p in full_text_paragraphs:
                            if 'VERDICT' in p.text:
                                full_text += p.text[:(p.text).index('VERDICT')]
                                break

                            else:
                                full_text += p.text + '\n'

                        if full_text == '':
                            full_text = finding_text.text

                    elif 'Is there a claim' in p.text:
                        full_text += p.text[:(p.text).index('Is there a claim')]
                        break 

                    elif finding_text is  None:
                        full_text += p.text + '\n'

                    clean_text = full_text.replace('\n', ' ').replace('\xa0', ' ')

        labels.append(label)
        claims.append(claim)
        textos.append(clean_text[:5])

    except Exception as e:
        textos_fallo += 1
        print("An exception occurred: " + str(e), i, label, claim, clean_text[:5])

        

An exception occurred: list index out of range https://dubawa.org/african-fact-checking-awards-opens-for-entry/ False The fact-checking organisation, Africa Check, has announced the opening of entries for the 10th edition of its African Fact-Checking Awards. As it
An exception occurred: list index out of range https://dubawa.org/fact-checking-apc-supporters-claim-on-nigerias-debt-servicing-in-2022/ False Claim: Nigeria spent N3.63 trillion on debt servicing between January and December 2022. React
An exception occurred: list index out of range https://dubawa.org/hoax-ecobank-is-not-running-any-subsidy-reward-scheme/ False Claim: A viral WhatsApp message says Ecobank rewards its clients in the “National Government with an $8000 subsidy.” Mr Ob
An exception occurred: list index out of range https://dubawa.org/no-evidence-police-colluded-with-thugs-inec-to-disrupt-rivers-state-election/ False CLAIM: Twitter users have shared a video claiming that police officers colluded with PDP thugs an

An exception occurred: list index out of range https://dubawa.org/true-image-posted-on-inecs-irev-for-ekiti-not-result-sheet/ True CLAIM: Social media users say the image uploaded on INEC’s IReV for Ekiti polling unit, not the result sheet. Follo
An exception occurred: list index out of range https://dubawa.org/no-election-results-for-all-states-yet-to-be-announced/ True CLAIM: Bashir Ahmad, Special Assistant on Digital Communications to Nigerian President Muhammadu Buhari, on his Twitter handle, claims that APC has won the elections in 14 states, PDP in 7, LP in 4 and NNPP in 1. Follo
An exception occurred: list index out of range https://dubawa.org/contrary-to-twitter-user-report-apc-not-lp-won-in-agege/ Misleading CLAIM: A Twitter user @drpenking tweeted an image that showed the Labour Party got 29,568 votes in Agege and claimed INEC switched the results to APC and reduced the numbers for Labour Party to 13,270. CLAIM
An exception occurred: 'list' object has no attribute 'capitalize

An exception occurred: list index out of range https://dubawa.org/top-ten-political-claims-we-fact-checked-in-2022/ Misleading The upcoming 2023 general elections have increased the rate of information disorder in Nigeria. Politicians and even citizens (with or without a motive) continue to put out information related to politics, especially on social media. The p
An exception occurred: list index out of range https://dubawa.org/no-statistics-to-support-tinubus-nigeria-youth-population-assertion/ Misleading Claim: Youths aged between 15 and 35 account for over 65 per cent or around 130 million of Nigeria’s 200 million population. The p
An exception occurred: list index out of range https://dubawa.org/ranking-most-ridiculous-claims-we-fact-checked-in-2022/ Misleading In a few days, the curtain will draw on 2022. As the world begins the countdown and looks forward to what 2023 will bring, a look back through 2022 brings memories and lessons to learn for many. The p
An exception occurred:

An exception occurred: list index out of range https://dubawa.org/how-true-is-claim-babies-can-be-grown-in-a-bag-instead-of-womb/ False Claim: A Facebook user claims babies can be grown in a bag. At th
An exception occurred: list index out of range https://dubawa.org/recurring-conspiracy-theory-claims-covid-19-vaccines-infect-users-with-hiv/ False Claim: A viral WhatsApp message and image links HIV to COVID-19 and smallpox vaccines At th
An exception occurred: list index out of range https://dubawa.org/video-shot-in-ghana-used-to-depict-support-for-atiku/ False Claim: A Twitter user claims youth in Adamawa State burnt brooms to show their support for Atiku Abubakar. At th
An exception occurred: list index out of range https://dubawa.org/the-central-bank-of-liberia-has-not-minted-coins-in-l25-and-l30-denominations/ False Claim: “The government through the Central Bank of (CBL) has minted coins in the L$25 and L$30 denominations” according to a Facebook user. At th
An exception occurred:

An exception occurred: list index out of range https://dubawa.org/yes-elon-musk-made-comment-about-purchasing-coca-cola-and-adding-cocaine/ False Claim: A viral screenshot claims Elon Musk in a tweet said he would buy Coca-Cola next and include cocaine as an ingredient. Clima
An exception occurred: list index out of range https://dubawa.org/nigerian-government-not-offering-national-youth-empowerment-fund/ Misleading Claim: A viral WhatsApp message claims the Federal Government is offering money to Nigerians under the National Youth Empowerment Fund. In a 
An exception occurred: list index out of range https://dubawa.org/is-lagos-largest-economy-in-africa-fact-checking-tinubus-claims-on-governorship-term/ Misleading In one year time, President Muhammadu Buhari will vacate office after eight years service as the number one citizen of Nigeria, and already, the race for who would take over from him in 2023 has commenced. In a 
An exception occurred: list index out of range https://dubawa.o

An exception occurred: list index out of range https://dubawa.org/is-ucj-unilorin-the-oldest-campus-news-outlet-in-nigeria/ False Claim: The Union of Campus Journalists (UCJ) at the University of Ilorin (UCJ UNILORIN) is the oldest campus news outlet in Nigeria. Many 
An exception occurred: list index out of range https://dubawa.org/is-burning-paper-soaked-in-honey-repulsive-for-snakes/ False Claim: Burning paper soaked in honey is repulsive to reptiles and will chase snakes away from houses and surroundings. After
An exception occurred: list index out of range https://dubawa.org/false-dr-ceesay-and-the-citizens-alliance-party-have-not-endorsed-adama-barrow/ False Claim: Dr. Ceesay and the Citizens Alliance party have endorsed Adama Barrow for this year’s presidential elections – social media posts. True 
An exception occurred: list index out of range https://dubawa.org/can-a-candidate-win-an-election-in-the-gambia-when-the-majority-of-the-voters-didnt-vote-for-them/ False Claim: A Twi

An exception occurred: list index out of range https://dubawa.org/is-ahmad-lawan-the-single-longest-serving-legislator-at-the-national-assembly/ False Claim: Lawyer and human rights activist Chidi Odinkalu, recently claimed that the Senate President, Sen. Ahmad Lawan, is the single longest-serving legislator at the National Assembly since 1999. Claim
An exception occurred: list index out of range https://dubawa.org/fani-kayode-shares-unrelated-photos-to-depict-military-onslaught-on-bandits-in-zamfara/ False Claim: Former Aviation Minister, Femi Fani-Kayode, recently shared some photos to celebrate what he claimed to be scenes of an onslaught on bandits in Zamfara and other northern states by the Nigerian army. Claim
An exception occurred: list index out of range https://dubawa.org/claim-dubai-was-built-with-12bn-infrastructure-fund-false/ False Claim: A Facebook user claimed that Dubai was built with a $12billion infrastructure fund. He also claimed that $2.4 billion was set up by Pres

In [405]:
len(labels)

435

In [50]:
print(len(labels), textos_fallo)

334 378


In [197]:
print(len(labels), len(claims), len(textos),textos_fallo)

297 297 297 22


In [378]:
(set(labels_list))

{'A',
 'According',
 'Also',
 'Although',
 'An',
 'Available',
 'Bola',
 'Both',
 'Checks',
 'Claim',
 'Contrary',
 'Dubawa',
 'Eating',
 'Even',
 'False',
 'False-',
 'Findings',
 'From',
 'Governor',
 'However',
 'Image',
 'Imf',
 'Inec',
 'Insufficient-evidence',
 'It',
 'Many',
 'Misleading',
 'More-context',
 'Most',
 'Mostly',
 'Mostly-false',
 'Mostly-true',
 'Nigeria',
 'No',
 'Official',
 'Our',
 'P',
 'Part',
 'Photo',
 'Photos',
 'Pictures',
 'President',
 'Prolonged',
 'Samuel',
 'Satire',
 'Taking',
 'The',
 'There',
 'These',
 'This',
 'Thunderstorm',
 'Timothy',
 'To',
 'True',
 'Twitter',
 'Unicef',
 'Verdict',
 'Verification',
 'Viral',
 'While',
 'Yahaya',
 'Yes'}

In [423]:
result = requests.get('https://dubawa.org/you-saw-video-of-this-ex-feminist-crying-begging-for-husband-its-satire/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
try:
    try:
        claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]
        #claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
    except IndexError:
        #claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]
        claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
    except:
        claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p').text.strip()))[0]

    verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')

    if verdict == [] or verdict is None or verdict.text[:7] == 'FINDING':

        try:
            label = re.findall(r'[VerdictVERDICT]?:\s(\w+)', (soup.find('p', class_ = 'has-vivid-red-background-color has-background').text))
        except:
            label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
            label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', label_raw[0].text, flags=re.IGNORECASE)[0]

        
        if label is None or label == []:
            obj = soup.find('figure', class_ = 'alignright size-full')
            if obj is None:
                obj = soup.find('figure', class_ = 'alignright size-large')
            try:
                label = re.findall(r'\w+\-?\w+', str(obj.img['data-image-title']))[0]
                if (label[-1:]).isdigit():
                    label = label[:-2]

            except:
                label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', label_raw[0].text, flags=re.IGNORECASE)[0]

        
    else: 
        try:
            label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', str((verdict.text).strip()), flags=re.IGNORECASE)[0]
            #label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
        except (AttributeError or IndexError): 
            verdict = soup.find('p', class_='has-white-background-color has-background')
            #label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
            label = re.findall(r'(TRUE|FALSE|MISLEADING|INSUFFICIENT-EVIDENCE|MOSTLY[-\s](?:TRUE|FALSE))', str((verdict.text).strip()), flags=re.IGNORECASE)[0]


    label = label.capitalize()
    if '-' in label:
        label.replace('-', ' ')

    text = soup.find('div', class_ = 'entry-content entry clearfix')

    try:
        full_text_heading = soup.find('h2', text=['Full Text', 'FULL TEXT', 'Full text'])

        full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 

        if (full_text_paragraphs == '') or (full_text_paragraphs == []):
            full_text_paragraphs = full_text_heading.find_all_next('p')
       

        full_text = ''
        for p in full_text_paragraphs:
            if 'The researcher' in p.text:
                full_text += p.text[:(p.text).index('The researcher')]
                break
            elif 'Is there a claim' in p.text:
                full_text += p.text[:(p.text).index('Is there a claim')]
                break 
            else:
                full_text += p.text + '\n'
    except:
            full_text = ''
            finding_text = None
            for p in soup.find_all('p'):

                if "FINDING:" in p.text:
                    finding_text = p
                    full_text_paragraphs = finding_text.find_all_next('p')

                    for p in full_text_paragraphs:
                        if 'VERDICT' in p.text:
                            full_text += p.text[:(p.text).index('VERDICT')]
                            break

                        else:
                            full_text += p.text + '\n'

                    if full_text == '':
                        full_text = finding_text.text

                elif 'Is there a claim' in p.text:
                    full_text += p.text[:(p.text).index('Is there a claim')]
                    break 

                elif finding_text is  None:
                    full_text += p.text + '\n'

                clean_text = full_text.replace('\n', ' ').replace('\xa0', ' ')

    print(label)
    
except Exception as e:
    textos_fallo += 1
    print("An exception occurred: " + str(e), label, claim, clean_text[:5])


An exception occurred: list index out of range False Claim: A TikTok video that has gone viral on WhatsApp in Nigeria shows a desperate feminist begging for a husband. Claim


In [287]:
pal = 'IGNORE-CASE'
'-' in pal
#pal.capitalize()

True

In [284]:
prueba = 'Verdict: FALSE. The highest inflation rate occurred in 1995, while former president Olusegun Obasanjo’s two-term tenure commenced in 1999 and ended in 2007.'

etiqueta = re.findall(r"Verdict:?\s+(\w+)", prueba, re.IGNORECASE)[0]
etiqueta

'FALSE'

In [ ]:
 #full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])
        
        #print(full_text, 'aquii')
        #text = (soup.find('div', class_ = 'entry-content entry clearfix')).text
        #lines = text.split("\n")
        #full_text_index = lines.index("Full Text") + 1  # add 1 to exclude the line with "Full Text"
        #text_after_full_text = "\n".join(lines[full_text_index:])
        #textos.append(full_text[:5])

In [103]:
result = requests.get('https://dubawa.org/image-from-tanzania-falsely-used-to-depict-boko-haram-attack/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
text = soup.find('div', class_ = 'entry-content entry clearfix')
#print(text)

#full_text_heading = text.find('h2', text=('FULL TEXT' or 'Full text' or 'Full Text'))
full_text_heading = soup.find('h2', text=['Full Text', 'FULL TEXT', 'Full text'])

#full_text_heading = text.find('h2') 
print(full_text_heading)

<h2><strong>Full Text</strong></h2>


In [41]:
labelss, claimss, textoss,  textos_fallo = [], [],[], 0
#headers = {'Accept-Language': 'en-US,en;q=0.8'}
result = requests.get('https://dubawa.org/akwa-ibom-kingdom-is-not-the-fifth-largest-natural-oil-reserve-in-africa/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
try:
    try:
        claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
    except:
        claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]

    verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')

    if verdict == [] or verdict is None or verdict.text[:7] == 'FINDING':
        obj = soup.find('figure', class_ = 'alignright size-full')
        if obj is None:
            label = re.findall(r'[VerdictVERDICT]?:\s|\b(\w+)', (soup.find('p', class_ = 'has-vivid-red-background-color has-background').text))[0]
        else: 
            try:
                label = re.findall(r'\w+\-?\w+', str(obj.img['data-image-title']))[0]
                if (label[-1:]).isdigit():
                    label = label[:-2]

            except:
                label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
                label = re.findall(r'[VERDICT]?:\s|\b(\w+)', (label_raw[0]).text) 

    else: 
        try:
            label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
        except (AttributeError or IndexError): 
            verdict = soup.find('p', class_='has-white-background-color has-background')
            label = re.findall(r'[VerdictVERDICTFINDING]?:\s*|\b(\w+)[!\.]?', str((verdict.text).strip()))[0]
    

    text = soup.find('div', class_ = 'entry-content entry clearfix')
    
    try:
        full_text_heading = text.find('h2', text='Full Text|FULL TEXT') 
        full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 
        full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])

        #text = (soup.find('div', class_ = 'entry-content entry clearfix')).text
        #lines = text.split("\n")
        #full_text_index = lines.index("Full Text") + 1  # add 1 to exclude the line with "Full Text"
        #text_after_full_text = "\n".join(lines[full_text_index:])
        #textos.append(full_text[:5])
    except: 
        finding_text = ""
        for p in soup.find_all('p'):

            if "FINDING:" in p.text:
                finding_text = p
                full_text_paragraphs = finding_text.find_all_next('p')
                full_text = ''
                for p in full_text_paragraphs:
                    if 'VERDICT' in p.text:
                        full_text += p.text[:(p.text).index('VERDICT')]
                        break
                    else:
                        full_text += p.text + '\n'
                            
        labelss.append(label)
        claimss.append(claim)
        textoss.append(full_text[:5])
      
    print(label, claim, verdict)
except:
    pass

#print(labelss, claimss, textos)

print(label, claim, full_text)
#print(full_text)

FALSE A user on an online community claims that Akwa Abasi Ibom kingdom is listed on Wikipedia as the fifth “highest” producer of oil and gas in Africa. 


In [48]:
result = requests.get('https://dubawa.org/akwa-ibom-kingdom-is-not-the-fifth-largest-natural-oil-reserve-in-africa/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
#claim = (re.findall(r'[ClaimCLAIM]?:\s|\b(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))
text = soup.find('div', class_ = 'entry-content entry clearfix')

print(text)
full_text_heading = text.find('h2', text=('FULL TEXT' or 'Full text'))
print(full_text_heading)
full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 
full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])
full_text


<div class="entry-content entry clearfix">
<p class="has-text-align-justify has-cyan-bluish-gray-background-color has-background"><strong>A user on an online community claims that Akwa Abasi Ibom kingdom is listed on Wikipedia as the fifth “highest” producer of oil and gas in Africa.</strong></p>
<div class="wp-block-image"><figure class="alignright size-large"><img alt="" class="wp-image-6352 jetpack-lazy-image" data-attachment-id="6352" data-comments-opened="1" data-image-caption="" data-image-description="" data-image-meta='{"aperture":"0","credit":"","camera":"","caption":"","created_timestamp":"0","copyright":"","focal_length":"0","iso":"0","shutter_speed":"0","title":"","orientation":"0"}' data-image-title="image-6" data-large-file="https://i0.wp.com/dubawa.org/wp-content/uploads/2020/10/image-6.png?fit=132%2C55&amp;ssl=1" data-lazy-src="https://i0.wp.com/dubawa.org/wp-content/uploads/2020/10/image-6.png?resize=132%2C55&amp;is-pending-load=1#038;ssl=1" data-medium-file="https://i

'On a discussion thread on Nairaland, a user by name Iluamoh claims that Akwa Abasi Ibom Kingdom lists on Wikipedia as the fifth “highest” producer of oil and gas in Africa after Nigeria, Angola, Algeria, and Libya. Nairaland is an online community and discussion forum for Nigerians.\nThe user also claims that Akwa Ibom State has the third-largest GDP in Nigeria.\xa0\nThe post appeared on Nairaland at 10.08 p.m. on August 19.\xa0\nThis nairalander, as members of the community are fondly called, suggests that with this ranking “Akwa Abasi Ibom Kingdom and Lagos will arguably be among the top ten biggest/richest countries in Africa, if they were to be independent countries of [it] own…”\xa0\nThis post garnered 688 views on Nairaland, a community of about 2,544,892 members.\nDubawa started verification with a google search engine quest which revealed that Akwa Abasi Kingdom does not exist.\xa0\nFurther, a review of crude oil production data by Trade Economics for August 2020 lists Nigeria

In [ ]:
text = soup.find('div', class_ = 'entry-content entry clearfix')
full_text_heading = text.find('h2', text='Full Text') 
full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 
full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])
        

        
text = (soup.find('div', class_ = 'entry-content entry clearfix')).text
lines = text.split("\n")
full_text_index = lines.index("Full Text") + 1  # add 1 to exclude the line with "Full Text"
text_after_full_text = "\n".join(lines[full_text_index:])

In [394]:
result = requests.get('https://dubawa.org/picture-of-students-sitting-on-mud-blocks-in-flooded-classroom-not-from-nigeria/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
text = (soup.find('div', class_ = 'entry-content entry clearfix')).text




Omoaka Christopher (@leroikris) tweeted a narrative with a rather disturbing picture of some students. They sat, on clay moulded blocks, in a cob building that is alleged to be their classroom. Hinged on the cracked-riddled walls of the structure appears to be their books packed in polythene bags. 
While their legs are stretched forward to avoid the flooded floor, their gloomy faces cast the hopelessness of their reality; one of the claimants said he would not wish such for the children of his enemies. 
Mr Christopher, who subtly implies the picture depicts the state of primary education in Nigeria, then suggests a solution, advising users to save the image as a reminder to vote wisely in the upcoming general elections in Nigeria, adding that Labour Party (LP) will guarantee a better Nigeria. 
“Just look at this picture. Would you even want your enemies’ children to be in classrooms of this nature? Save it on your phone as a reminder when you go to the polling station on 25 February 20

In [356]:
len(labels), textos_fallo

(242, 242)

In [318]:
result = requests.get('https://dubawa.org/apc-national-womens-leader-betta-edu-shares-misleading-claims-about-elections-in-cross-river-state/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
text = soup.find('div', class_ = 'entry-content entry clearfix')
full_text_heading = text.find('h2', text='Full Text') 
full_text_paragraphs = full_text_heading.find_all_next('p', class_='has-text-align-justify') 
full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])
full_text

''

In [319]:
result = requests.get('https://dubawa.org/apc-national-womens-leader-betta-edu-shares-misleading-claims-about-elections-in-cross-river-state/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
text = soup.find('div', class_ = 'entry-content entry clearfix')
full_text_heading = text.find('h2', text='Full Text')
full_text_paragraphs = full_text_heading.find_all_next('p') 
#full_text = '\n'.join([p.get_text() for p in full_text_paragraphs])
full_text = ''
for p in full_text_paragraphs:
    if 'The researcher' in p.text:
        full_text += p.text[:(p.text).index('The researcher')]
        break
    else:
        full_text += p.text + '\n'
full_text





'In a recent interview on Arise TV, a former commissioner of health in Cross River state and the current National  Women Leader of the ruling All Progressive Congress, APC made various claims regarding the recent elections. Speaking about the national parliamentary elections, Ms Edu said:\n“I want to put this to the fore that APC won the House of Representative seats except that of the Northern part of the state. So, APC won in the central, APC won in the south, and we only lost in the Northern part of the state.”\xa0\nOn the state governorship elections, she said:\n“The Labour Party won [the presidential election in the state], APC came close second, and then PDP came far third. Going into the next election, Labour Party doesn’t have a candidate.” Ms Betta added, “for PDP, we are not worried about them. They are not a threat to us. We are discussing with them and working hand in hand with them because they do not even have a governorship candidate. They will work with us in APC.”\xa0\

In [353]:
result = requests.get('https://dubawa.org/true-voting-yet-to-commence-in-parts-of-bayelsa-at-4-pm/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')
verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')

if verdict == [] or verdict is None or verdict.text[:7] == 'FINDING':
    obj = soup.find('figure', class_ = 'alignright size-full')
    try:
        label = re.findall(r'\w+\-?\w+', str(obj.img['data-image-title']))[0]
        if (label[-1:]).isdigit():
            label = label[:-2]
        #print(label, '')
    except:
        label_raw = (soup.find_all('p', class_ = 'has-background'))[-1:]
        label = re.findall(r'VERDICT:\s(\w+)', (label_raw[0]).text)
        
else: 
    try:
        label = re.findall(r'[VerdictVERDICTFINDING]:\s*(\w+)[!\.]?', str((verdict.text).strip()))[0]
    except (AttributeError or IndexError): 
        verdict = soup.find('p', class_='has-white-background-color has-background')
        label = re.findall(r'[VerdictVERDICTFINDING]:\s*(\w+)[!\.]?', str((verdict.text).strip()))[0]

print(label)


['CORRECT']


In [220]:
prueba = 'Verdict: False. Online research on the particular Press Conference revealed that Morlu did not mention such a statement. The ruling party chairman also openly clarified the allegation to the media.'
label = re.findall(r'[VerdictVERDICT]:\s*(\w+)[!\.]', prueba)

label

['False']

In [151]:
result = requests.get('https://dubawa.org/has-inec-uploaded-results-of-national-assembly-elections-on-irev-as-claimed/')
src = result.content     
soup = BeautifulSoup(src, 'html.parser')

try:
    claim = (re.findall(r'Claim|CLAIM:\s(.+)',(soup.find('p', class_ = 'has-text-align-justify has-cyan-bluish-gray-background-color has-background').text).strip()))[0]
    
except:
    claim = (re.findall(r'Claim|CLAIM:\s(.+)',soup.find('p', class_ = 'has-cyan-bluish-gray-background-color has-background').text.strip()))[0]
print(claim)

INEC has uploaded National Assembly election results on the IReV portal but has yet to do so for presidential elections.


In [104]:
df = pd.DataFrame.from_records(res_list, columns=["Claim", "Evidences", "Label", "Date", "Url"])
df

,Claim,Evidences,Label,Date,Url
0,"Nigeria’s Minister of Finance, Zainab Ahmed, r...",As Nigeria struggles with a rising cost of gov...,False,03/04/2023,https://dubawa.org/nigerias-finance-minister-l...
1,"The family of Justice Gloria Musu-Scott, in a ...",The murder and armed robbery attacks on the ho...,False,22/03/2023,https://dubawa.org/false-no-evidence-to-suppor...
2,A Twitter user alleged that a 19-year-old boy ...,A suicide photo of a young man has surfaced on...,False,21/03/2023,https://dubawa.org/picture-of-boy-who-committe...
3,"Nigerian professor and Dan David Prize winner,...",Nigerian creatives typically perform well in g...,FALSE,16/03/2023,https://dubawa.org/parody-account-holder-parad...
4,Viral news articles published by “The Indiana...,"Two popular news outlets in Asia, namely, The ...",False,15/03/2023,https://dubawa.org/bengali-bangla-has-never-be...
5,“Viral image of a lady smoking hookah is the d...,As the rescheduled governorship election in La...,FALSE,15/03/2023,https://dubawa.org/viral-image-of-hookah-smoki...


In [163]:
df2 = pd.DataFrame.from_records(res_list, columns=["Claim", "Evidences", "Label", "Date", "Url"])
df2

,Claim,Evidences,Label,Date,Url
0,CLAIM: INEC has uploaded National Assembly ele...,Reactions worldwide have followed the February...,FALSE,04/04/2023,https://dubawa.org/has-inec-uploaded-results-o...
1,Claim: The bacteria in yoghurt is the same as ...,"Yoghurt, the creamy and tangy dairy delight, h...",FALSE,03/04/2023,https://dubawa.org/are-bacteria-found-in-yoghu...
2,Claim: A viral WhatsApp message claims that th...,"Worldwide, Muslims start their annual fast per...",FALSE,03/04/2023,https://dubawa.org/dubai-government-not-giving...
3,"Claim: Pastor Enoch Adeboye, the General Overs...","On February 21, 2023, a Twitter user with the ...",MISLEADING,03/04/2023,https://dubawa.org/did-adeboye-lead-protest-ag...
4,Claim: “The purported letter from U.S Ambassad...,Liberians residing in the United States who we...,FALSE,03/04/2023,https://dubawa.org/us-envoys-comment-about-dis...
...,...,...,...,...,...
388,Claim: A picture appeared recently on both Wha...,Claim: A picture appeared recently on both Wha...,INSUFFICIENT-EVIDENCE,17/08/2021,https://dubawa.org/does-eating-food-in-plastic...
389,Claim: Boy’s skin peeling off after he was bit...,Claim: Boy’s skin peeling off after he was bit...,INSUFFICIENT-EVIDENCE,17/08/2021,https://dubawa.org/no-evidence-childs-severe-s...
390,"Claim: a recent Facebook post, Dapo Abiodun, t...","Claim: a recent Facebook post, Dapo Abiodun, t...",TRUE,17/08/2021,https://dubawa.org/records-show-theophilus-ogu...
391,Claim: Sucking a pregnant woman’s breast prepa...,"On August 5, 2021, a Chief Nursing Sister, Ros...",TRUE,12/08/2021,https://dubawa.org/true-sucking-pregnant-woman...


In [331]:
df3 = pd.DataFrame.from_records(res_list, columns=["Claim", "Evidences", "Label", "Date", "Url"])
df3

,Claim,Evidences,Label,Date,Url
0,CLAIM: INEC has uploaded National Assembly ele...,Reactions worldwide have followed the February...,False,04/04/2023,https://dubawa.org/has-inec-uploaded-results-o...
1,Claim: The bacteria in yoghurt is the same as ...,"Yoghurt, the creamy and tangy dairy delight, h...",False,03/04/2023,https://dubawa.org/are-bacteria-found-in-yoghu...
2,Claim: A viral WhatsApp message claims that th...,"Worldwide, Muslims start their annual fast per...",False,03/04/2023,https://dubawa.org/dubai-government-not-giving...
3,"Claim: Pastor Enoch Adeboye, the General Overs...","On February 21, 2023, a Twitter user with the ...",Misleading,03/04/2023,https://dubawa.org/did-adeboye-lead-protest-ag...
4,Claim: “The purported letter from U.S Ambassad...,Liberians residing in the United States who we...,False,03/04/2023,https://dubawa.org/us-envoys-comment-about-dis...
...,...,...,...,...,...
683,"Claim: a viral Mobile message posing as INEC, ...","Claim: a viral Mobile message posing as INEC, ...",False,15/10/2020,https://dubawa.org/quick-checks-ondo-2020-elec...
684,Claim: APC asks court to disqualify Akeredolu ...,Claim: APC asks court to disqualify Akeredolu ...,Verdict,15/10/2020,https://dubawa.org/quick-checks-ondo-2020-elec...
685,The Deputy National Publicity Secretary of the...,After a series of protests against the activit...,False,14/10/2020,https://dubawa.org/sars-has-been-in-existence-...
686,A user on an online community claims that Akwa...,"On a discussion thread on Nairaland, a user by...",False,21/09/2020,https://dubawa.org/akwa-ibom-kingdom-is-not-th...


In [3]:
df4 = pd.DataFrame.from_records(res_list, columns=["Claim", "Evidences", "Label", "Date", "Url"])
df4

NameError: name 'pd' is not defined

In [368]:
print(df3.iloc[684]['Label'])
df3.iloc[684]['Url']
#df.loc[2, 'col1'] = new_val

Verdict


'https://dubawa.org/quick-checks-ondo-2020-election/'

In [394]:
labels_list = df4['Label'].tolist()
(set(labels_list))

{'False',
 'False-',
 'Misleading',
 'Mostly false',
 'Mostly true',
 'True',
 'Verdict'}

In [396]:
set(fallos)

{'https://dubawa.org/10-photos-used-to-depict-obigbo-victims-are-from-osun-lagos-others/',
 'https://dubawa.org/2016-video-recirculated-to-depict-nigerian-army-killed-dozens-of-bandits-in-kebbi/',
 'https://dubawa.org/2018-photograph-recycled-as-recent-military-attack-in-abia-state/',
 'https://dubawa.org/2018-video-circulated-to-sell-narrative-two-survived-plane-crash-that-killed-nigerias-army-chief/',
 'https://dubawa.org/2019-report-recirculated-to-fuel-atiku-buhari-presidential-election-rivalry-and-many-fell-for-it/',
 'https://dubawa.org/2019-videos-recirculated-to-depict-massive-turnout-for-tinubu-in-katsina/',
 'https://dubawa.org/2023-elections-are-there-invalid-thumbprinting-as-reported-in-this-post/',
 'https://dubawa.org/a-post-on-ussd-codes-to-lock-bank-accounts-went-viral-recently-we-checked-if-they-are-functional/',
 'https://dubawa.org/a-twitter-influencer-alleged-that-some-european-protesters-recently-harassed-godwin-obaseki/',
 'https://dubawa.org/abba-kyari-not-in-aus

In [335]:
df3.iloc[687]['Url']

'https://dubawa.org/nigeria-not-ranked-third-most-terrorised-country-in-2020/'

In [254]:
df3.to_csv('Dubawa.csv', index=False)

In [105]:
result = requests.get('https://dubawa.org/liberian-finance-ministers-affirmation-of-us50-per-bag-of-rice-price-in-sierra-leone-cannot-be-substantiated/', headers=headers)
src = result.content     
soup = BeautifulSoup(src, 'html.parser')  
obj = soup.find('figure', class_ = 'alignright size-full')
image_title = obj.img['data-image-title']
image_title

'INSUFFICIENT-EVIDENCE'

In [95]:
verdict = soup.find('p', class_ = 'has-text-align-justify has-white-color has-text-color has-background')
#label = re.findall(r'(\w+)\.', str(verdict))[0]
veredict == []

True

In [6]:

import pandas as pd 
dataframe = pd.read_csv('Dubawa.csv')

In [10]:
(dataframe['Label'].unique())

array(['FALSE', 'MISLEADING', 'Verdict', 'VERDICT',
       'INSUFFICIENT-EVIDENCE', 'MOSTLY-TRUE', 'TRUE', 'FALSE-',
       'CORRECT', 's', 'is', 'State', 'started', '2007', 'scorecards',
       'public', "['False']", 'MOSTLY-FALSE', 'Daily', 'point',
       'incident', 'closed', 'Emefiele', 'claim', 'activities', 'genuine',
       'Africa', 'false', 'poisonous', 'days', 'image', 'noon', 'true',
       'it', 'trips', '2022', 'billion', 'misleading', 'election', 'True',
       'debunked', '[]', 'sperm', 'IGP', 'identification', 'cancer',
       'destruction', 'risk', 'MORE-CONTEXT', 'SATIRE', 'visa',
       'customers', 'website', 'Kyari', 'Abuja', '2019', 'fake', 'forces',
       'Mubarak', 'phony', 'casualties', 'TikTok', 'Nigeria',
       'manipulated', 'so', 'Tangwa', 'Minister', 'female', '2015',
       'proven', '2021', 'message', 'claims', 'fraudulent', 'Wrong',
       'festival', 'spark', 'liver', 'Harcourt', 'Monotechnics', 'scam',
       'this', 'news', 'Igbokwe', 'USA', 'inco